## Preliminaries

In [1]:
# To auto-reload modules in jupyter notebook (so that changes in files *.py doesn't require manual reloading):
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
%load_ext autoreload
%autoreload 2

Import commonly used libraries and magic command for inline plotting

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
%matplotlib inline

## Analysis 2 – Wrangling csv files

# Step 1 Consolidation

Imports

In [28]:
import pandas as pd
from openpyxl import Workbook
from pathlib import Path

Step 1: Create a new Excel file

In [29]:
excel_filename = 'BCM.xlsx'
workbook = Workbook()  # This creates a new workbook with a default sheet named 'Sheet'
workbook.save(excel_filename)

Define the directory where your CSV files are stored


In [30]:
csv_directory = 'logs'  # Make sure to set this to the correct path

Use pathlib to glob through the directory and find all CSV files


In [31]:
csv_files = Path(csv_directory).glob('*.csv')

Lets loop through each csv now

In [32]:
headers = ['datetime', 'scale', 'temperature']
for csv_file in csv_files:
    try:
        # Read the CSV file into a pandas DataFrame
        data = pd.read_csv(csv_file, header=None)  # Specify that there is no header row
        data.columns = headers  # Assign the headers to the DataFrame

        # Insert the DataFrame into a new sheet in the Excel file
        # Use the name of the CSV file as the sheet name
        sheet_name = csv_file.stem  # This gets the file name without the extension
        with pd.ExcelWriter(excel_filename, engine='openpyxl', mode='a') as writer:  # Open the file in append mode
            data.to_excel(writer, sheet_name=sheet_name, index=False)  # Write the data to a new sheet
    except Exception as e:
        print(f"An error occurred processing file {csv_file}: {e}")

# Notify when the task is complete
print("All CSV files have been inserted into the Excel file.")


All CSV files have been inserted into the Excel file.


# Step 2 – Summarization

Similar to above but now we will be adding to the .xlsx

Starting with our imports

In [33]:
import pandas as pd
from openpyxl import Workbook, load_workbook
from pathlib import Path

Step 1: Create a new Excel file


In [34]:
excel_filename = 'BCM.xlsx'
workbook = Workbook()  # This creates a new workbook with a default sheet named 'Sheet'
workbook.save(excel_filename)

Define the directory where your CSV files are stored


In [35]:
csv_directory = 'logs'  # Make sure to set this to the correct path

# Use pathlib to glob through the directory and find all CSV files
csv_files = Path(csv_directory).glob('*.csv')

Loop through each csv but this time we're adding labels and formulas

In [36]:
# Headers
headers = ['datetime', 'scale', 'temperature']

# Loop through each CSV file
for csv_file in csv_files:
    try:
        # Read the CSV file into a pandas DataFrame
        data = pd.read_csv(csv_file, header=None)  # Specify that there is no header row
        data.columns = headers  # Assign the headers to the DataFrame

        # Convert the 'datetime' column to pandas datetime
        data['datetime'] = pd.to_datetime(data['datetime'], infer_datetime_format=True)

        # Insert the DataFrame into a new sheet in the Excel file
        # Use the name of the CSV file as the sheet name
        sheet_name = csv_file.stem  # This gets the file name without the extension
        with pd.ExcelWriter(excel_filename, engine='openpyxl', mode='a', datetime_format='yyyy-mm-dd hh:mm:ss') as writer:
            data.to_excel(writer, sheet_name=sheet_name, index=False)  # Write the data to a new sheet
        
        # Reload the workbook to access the newly created sheet
        wb = load_workbook(excel_filename)
        ws = wb[sheet_name]

        # Add the labels and formulas
        ws['G2'] = 'Min Temperature'
        ws['G3'] = 'Max Temperature'
        ws['G4'] = 'Average Temperature'
        ws['G6'] = 'Min Datetime'
        ws['G7'] = 'Max Datetime'

        last_row = ws.max_row
        ws['H2'] = f'=MIN(C2:C{last_row})'
        ws['H3'] = f'=MAX(C2:C{last_row})'
        ws['H4'] = f'=AVERAGE(C2:C{last_row})'
        ws['H6'] = f'=MIN(A2:A{last_row})'
        ws['H7'] = f'=MAX(A2:A{last_row})'
        
        # Apply custom datetime format to min and max datetime cells
        ws['H6'].number_format = 'YYYY-MM-DD HH:MM:SS'
        ws['H7'].number_format = 'YYYY-MM-DD HH:MM:SS'

        # Apply cell formatting to column H
        for row in ws['H2:H4']:
            for cell in row:
                cell.number_format = '0.00'
        
        # Save the changes to the workbook
        wb.save(excel_filename)
    except Exception as e:
        print(f"An error occurred processing file {csv_file}: {e}")

# Notify when the task is complete
print("All CSV files have been inserted into the Excel file.")


All CSV files have been inserted into the Excel file.
